# SnorkelMED - Identifying Opioid-Induced Respiratory Depression  

The purpose of this analysis is to probabilistically identify which patient visits included an opioid-induced respiratory depression (OIRD) event. In this notebook, we use our LFs and the validation set to develop the deterministic model. 

In [ ]:
%matplotlib inline
import os
import re
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns

from snorkel.labeling import labeling_function, PandasLFApplier, LFAnalysis, filter_unlabeled_dataframe
from snorkel.labeling.model import MajorityLabelVoter, LabelModel
from snorkel.analysis import get_label_buckets, metric_score
from snorkel.utils import probs_to_preds

majority_model = MajorityLabelVoter()
label_model = LabelModel(cardinality=2, verbose=True)

import helper as hlp
import importlib
importlib.reload(hlp)

# global variables
ABSTAIN = -1; CONTROL = 0; CASE = 1

In [ ]:
# load updated training/dev/valid data after labeling from previous round
df_train, df_dev, df_valid, df_test = hlp.load_data(round=5)

# re-attach numeric data to reflect any updated rules
df_train, df_dev, df_valid, df_test = hlp.reattach_numeric_data(df_train, df_dev, df_valid, df_test)

# keep confounding diagnoses visits available
confounding_diagnosis_present = pd.read_csv('../sd_structured/icd/visits_with_confounding_icd_codes.csv')

# made changes in code in to loading confounding diagnoses, so eliminating the redundant columns in corrected
#    validation set
df_valid.drop(['condition_start_date', 'cva', 'sepsis'], axis = 1, inplace = True)

In [ ]:
# store Y values for ease of evaluation
Y_dev = df_dev['label'].values
Y_dev = np.where(Y_dev=='case', 1, 0) 

Y_valid = df_valid['label'].values
Y_valid = np.where(Y_valid=='case', 1, 0) 

## Apply Generative Labels to Validation Set

In [ ]:
# copied directly from last round of learning function applications
# removes LFs that were ultimately dropped
lfd = dict()

@labeling_function()
def LF_naloxone_admin(x):
    if x['naloxone_admin_prob'] >= 0.8:
        if x['counts_naloxone_NOT_effective'] > x['counts_naloxone_effective']:
            return CONTROL
        elif x['counts_naloxone_effective'] > 0:
            return CASE
        elif x['counts_naloxone_NOT_effective'] > 0:
            return CONTROL
        else:
            return CASE
    elif x['naloxone_admin_prob'] < 0.8:
        return CONTROL
    else:
        return CONTROL
lfd['LF_naloxone_admin'] = 0

@labeling_function()
def LF_counts_naloxone(x):
    if x['counts_naloxone'] > 0: 
        if x['counts_naloxone_NOT_effective'] > x['counts_naloxone_effective']:
            return CONTROL
        elif x['counts_naloxone_effective'] > 0:
            return CASE
        elif x['counts_naloxone_NOT_effective'] > 0:
            return CONTROL
        else:
            return CASE
    return ABSTAIN
lfd['LF_counts_naloxone'] = max(lfd.values()) + 1

@labeling_function()
def LF_respiratory_failure_any(x):
    if '1' in x['respiratory_failure_any'].lower(): 
        if 'yes' in x['eligible_vent'].lower():
            return CONTROL
    return ABSTAIN
lfd['LF_respiratory_failure_any'] = max(lfd.values()) + 1

@labeling_function()
def LF_counts_resp_care_notes(x):
    if x['counts_resp_care_notes'] == 0:
        return CONTROL
    return ABSTAIN
lfd['LF_counts_resp_care_notes'] = max(lfd.values()) + 1

@labeling_function()
def LF_counts_no_acute_events(x):
    if x['counts_no_acute_events'] > 0:
        if x['counts_naloxone'] + x['counts_rapid_response'] + x['counts_altered_mental_status'] + \
            x['counts_narcotic_overdose'] + x['counts_hypoxia'] + x['counts_held_opioids'] > 0:
            return ABSTAIN
        return CONTROL
    return ABSTAIN
lfd['LF_counts_no_acute_events'] = max(lfd.values()) + 1

@labeling_function()
def LF_counts_altered_mental_status(x):
    if x['counts_altered_mental_status'] > 0:
        if x['visit_occurrence_id'] in confounding_diagnosis_present['visit_occurrence_id'].unique(): 
            return ABSTAIN
        return CASE
    return ABSTAIN
lfd['LF_counts_altered_mental_status'] = max(lfd.values()) + 1

@labeling_function()
def LF_counts_narcotic_overdose(x):
    if x['counts_narcotic_overdose'] > 0:
        return CASE
    return ABSTAIN
lfd['LF_counts_narcotic_overdose'] = max(lfd.values()) + 1

@labeling_function()
def LF_counts_hypoxia(x):
    if x['counts_hypoxia'] > 0:
        return CASE
    return ABSTAIN
lfd['LF_counts_hypoxia'] = max(lfd.values()) + 1

@labeling_function()
def LF_counts_decrease_opioids(x):
    if x['counts_decrease_opioids'] > 0:
        return CASE
    return ABSTAIN
lfd['LF_counts_decrease_opioids'] = max(lfd.values()) + 1

@labeling_function()
def LF_extended_vent_time(x):
    if 'yes;;yes;;yes;;yes' in x['eligible_vent'].lower(): 
        return CONTROL
    return ABSTAIN
lfd['LF_extended_vent_time'] = max(lfd.values()) + 1

@labeling_function()
def LF_counts_no_pain_meds(x):
    if x['counts_no_pain_meds'] > 0:
        return CONTROL
    return ABSTAIN
lfd['LF_counts_no_pain_meds'] = max(lfd.values()) + 1

@labeling_function()
def LF_no_support(x):
    if x['counts_naloxone'] == 0 and \
        np.isnan(x['naloxone_admin_prob']) and \
        x['counts_altered_mental_status'] == 0 and \
        x['counts_narcotic_overdose'] == 0 and \
        x['counts_hypoxia'] == 0 and \
        x['counts_decrease_opioids'] == 0 and \
        x['counts_rapid_response'] == 0 and \
        x['counts_held_opioids'] == 0 and \
        x['counts_pinpoint_pupils'] == 0 and \
        '1' not in x['respiratory_failure_any'].lower():
        return CONTROL
    return ABSTAIN
lfd['LF_no_support'] = max(lfd.values()) + 1

@labeling_function()
def LF_counts_held_opioids(x):
    if x['counts_held_opioids'] > 0:
        return CASE
    return ABSTAIN
lfd['LF_counts_held_opioids'] = max(lfd.values()) + 1

@labeling_function()
def LF_counts_pinpoint_pupils(x):
    if x['counts_pinpoint_pupils'] > 0:
        return CASE
    return ABSTAIN
lfd['LF_counts_pinpoint_pupils'] = max(lfd.values()) + 1

In [ ]:
lfs = [LF_naloxone_admin, LF_counts_naloxone, LF_respiratory_failure_any, LF_counts_resp_care_notes,
       LF_counts_no_acute_events, LF_counts_altered_mental_status, LF_counts_narcotic_overdose,
       LF_counts_hypoxia, LF_counts_decrease_opioids, LF_extended_vent_time, LF_counts_no_pain_meds,
       LF_no_support, LF_counts_held_opioids, LF_counts_pinpoint_pupils]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)
L_dev = applier.apply(df=df_dev)
L_valid = applier.apply(df=df_valid)
L_test = applier.apply(df=df_test)

In [ ]:
# combine train & dev sets into new training set
df_train_dev = df_train.append(df_dev, sort=False)

L_train_dev = applier.apply(df=df_train_dev)

In [ ]:
# best hyper-parameters from most current LF notebook
n_epochs = 2000
lr = 0.01 
lr_scheduler = 'step'
seed = 987 
class_balance = [0.985, 0.015] # major class imabalance (5-15 events per 1,000 surgical cases, so 0.005-0.015)

# replace Y_dev with Y_valid in this notebook
label_model.fit(L_train = L_train_dev, Y_dev = Y_valid, n_epochs = n_epochs, lr = lr, optimizer = 'adamax', 
                lr_scheduler = lr_scheduler, log_freq = 100, seed = seed, 
                class_balance = class_balance)

In [ ]:
# FYI, these numbers will look slightly different because there are 20 fewer patients in the training set
#    after removing the top-scoring patients & placing in dev & valid sets, and then the 90-patient dev
#    sets get appended back to the train set
#    But the numbers look pretty similar to before
LFAnalysis(L=L_train_dev, lfs=lfs) \
    .lf_summary(est_weights = label_model.get_weights()) \
    .sort_values(by='Learned Weight', ascending=False)

In [ ]:
# empirical accuracy within Validation set
LFAnalysis(L=L_valid, lfs=lfs).lf_summary(Y=Y_valid).sort_values('Emp. Acc.', ascending=False)

In [ ]:
def performance_lf(L, Y):
    """ Helper function for printing evaluation of performance. """
    for model in ['maj', 'lab']:
        if model == 'maj':
            print("Majority Vote...")
            acc = majority_model.score(L=L, Y=Y)['accuracy']
            probs = majority_model.predict_proba(L)
        elif model == 'lab':
            print("Label Model...")
            acc = label_model.score(L=L, Y=Y)['accuracy']
            probs = label_model.predict_proba(L)
            
        preds = probs_to_preds(probs)
        print(f"{'Accuracy:':<10} {acc * 100:.1f}%")
        print(f"{'F1 score:':<10} {metric_score(Y, preds, probs=probs, metric='f1') * 1:.3f}")
        print(f"{'AUC:':<10} {metric_score(Y, preds, probs=probs, metric='roc_auc') * 1:.3f}", '\n')

In [ ]:
performance_lf(L_dev, Y_dev)

The label model outperforms the majority vote in all metrics for the dev set, but this is also over-fit since we included in the dev set in the train set now. 

In [ ]:
performance_lf(L_valid, Y_valid)

The majority vote does a bit better on the validation set, with respect to accuracy & F1 score. The AUC is higher in the Label Model. 

In [ ]:
# assign probabilities from either majority vote or label model
gen_probs_train = majority_model.predict_proba(L=L_train_dev)
gen_probs_dev = majority_model.predict_proba(L=L_dev)
gen_probs_valid = majority_model.predict_proba(L=L_valid)

hlp.plot_probabilities_histogram(gen_probs_valid[:, CASE])

In [ ]:
gen_probs_train = label_model.predict_proba(L=L_train_dev)
gen_probs_dev = label_model.predict_proba(L=L_dev)
gen_probs_valid = label_model.predict_proba(L=L_valid)

hlp.plot_probabilities_histogram(gen_probs_valid[:, CASE])

In [ ]:
# how do these compare? 
maj = majority_model.predict_proba(L=L_valid)
lab = label_model.predict_proba(L=L_valid)

plt.scatter(lab, maj)

## Export Relevant Data for Analysis

In [ ]:
export_data = (df_train_dev, df_valid, df_test, Y_dev, Y_valid, 
               L_train, L_dev, L_train_dev, L_valid, 
               label_model, majority_model, L_test)

with open('data_for_analysis.pkl', 'wb') as f:
    pickle.dump(export_data, f)